In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

2.9.2


## Fashin Mnist 불러오기

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


## 데이터 차원 확인, 스케일링

In [3]:
train_x=train_x/255.0
test_x=test_x/255.0

In [4]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
# 그림 한 장을 3차원으로 표현 (채널의 개념), 총 60000장의 그림
train_x = train_x.reshape(60000, 28, 28, 1) # 마지막 1 : 채널 (컬러 데이터였다면 rgb를 나타내는 3)
train_x.shape

(60000, 28, 28, 1)

In [6]:
test_x = test_x.reshape(10000, 28, 28, 1)
test_x.shape

(10000, 28, 28, 1)

## Convolution Layer

In [7]:
l = layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu') # input_shape : 한 장의 사진에 대한 shape

In [8]:
train_x[:10].shape

(10, 28, 28, 1)

In [9]:
l(train_x[:10]) # 10장만 넣어보기

<tf.Tensor: shape=(10, 26, 26, 32), dtype=float32, numpy=
array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         

In [10]:
l(train_x[:10]).shape

TensorShape([10, 26, 26, 32])

한 장의 사진에 대해 3,3 커널이 32개 지나가면 26, 26 사이즈의 32개 feature map이 나옴  
총 10장의 사진을 넣었으니 shape은 [10, 26, 26, 32]

In [11]:
# l_stride_2 = layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu', strides=2)
# l_stride_2(train_x[:10]).shape

In [12]:
p = layers.MaxPool2D(input_shape=(26, 26, 32), pool_size=(2, 2))
p(l(train_x[:10])).shape

TensorShape([10, 13, 13, 32])

## CNN 만들기

In [13]:
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)), # 여기서부터 input shape는 알아서 계산해줌
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5), # 과적합 방지를 위해 절반만 활성화
    layers.Dense(10, activation='softmax') # 10개 중에 하나로 분류
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

`2_ImageBasicClassification_Fashion_MNIST` 의 Fully Connected Neural Network 모델과 파라미터 수 비교

In [14]:
## 모형 컵파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
## 조기중단 콜백
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [16]:
model.fit(train_x, train_y, batch_size=64, epochs=1000, callbacks=[early_stop], validation_split=0.2)

Epoch 1/1000
750/750 [==============================] - 6s 5ms/step - loss: 0.6242 - accuracy: 0.7747 - val_loss: 0.4239 - val_accuracy: 0.8542
Epoch 2/1000
750/750 [==============================] - 5s 7ms/step - loss: 0.4246 - accuracy: 0.8472 - val_loss: 0.3615 - val_accuracy: 0.8738
Epoch 3/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.3774 - accuracy: 0.8638 - val_loss: 0.3365 - val_accuracy: 0.8813
Epoch 4/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.3523 - accuracy: 0.8727 - val_loss: 0.3220 - val_accuracy: 0.8832
Epoch 5/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.3302 - accuracy: 0.8802 - val_loss: 0.2992 - val_accuracy: 0.8908
Epoch 6/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.3169 - accuracy: 0.8855 - val_loss: 0.2934 - val_accuracy: 0.8939
Epoch 7/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.3056 - accuracy: 0.8887 - val_loss: 0.2835 - val_accuracy:

In [17]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 3ms/step - loss: 0.2516 - accuracy: 0.9151


[0.25163137912750244, 0.9150999784469604]

`2_ImageBasicClassification_Fashion_MNIST` 의 Fully Connected Neural Network 모델과 성능 비교